In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
# scale = 1/5 
# scale = 1/10
# scale = 1/20 
# scale = 1/40

In [3]:
# def tf_aug(X_in, scaler=scale):
#     X_out = np.zeros_like(X_in)
#     for channel in range(X_in.shape[-1]):
#         this_X = X_in[..., channel][np.newaxis, ..., np.newaxis]
#         scale = np.abs(np.max(this_X) - np.min(this_X)) * scaler
#         this_X = jitter(this_X, sigma=scale)
#         this_X = scaling(this_X, sigma=scale)
#         this_X = time_warp(this_X, sigma=scale)
#         this_X = window_warp(this_X, window_ratio=scale)
#         X_out[..., channel] = this_X[0, ..., 0]
#     return X_out


# def augment_this(X_in, y_in):
#     X_out = tf.numpy_function(tf_aug, [X_in], tf.float32)    
#     return X_out, y_in


In [4]:
X, y, site = load_data()

2022-06-29 14:29:45.934861: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
from sklearn.utils import shuffle, resample

In [6]:
from sklearn.metrics import r2_score, median_absolute_error

In [7]:
model_dict = {
  "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  # "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [8]:
n_runs = 10

In [9]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [10]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [11]:
dfs_eval = []
dfs_pred = []

In [12]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii])
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_1_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_1_pred.csv")

##################################################
model:  cnn_lenet
run:  0
pooling layers: 4
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 6)            1302      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 50, 6)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 16)            304       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 25, 16)           0         
 1D)                                                             
                                

Epoch 21/1000
 7/10 [====================>.........] - ETA: 0s - loss: 16.3996 - mean_squared_error: 16.3996 - rmse: 4.0496 - mean_absolute_error: 3.1661
Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
10/10 [==============================] - 0s 23ms/step - loss: 16.2016 - mean_squared_error: 16.2016 - rmse: 4.0251 - mean_absolute_error: 3.1336 - val_loss: 9.9788 - val_mean_squared_error: 9.9788 - val_rmse: 3.1589 - val_mean_absolute_error: 2.4120 - lr: 0.0010
Epoch 22/1000
10/10 [==============================] - 0s 21ms/step - loss: 17.4108 - mean_squared_error: 17.4108 - rmse: 4.1726 - mean_absolute_error: 3.2990 - val_loss: 10.5169 - val_mean_squared_error: 10.5169 - val_rmse: 3.2430 - val_mean_absolute_error: 2.4515 - lr: 5.0000e-04
Epoch 23/1000
10/10 [==============================] - 0s 22ms/step - loss: 16.2391 - mean_squared_error: 16.2391 - rmse: 4.0298 - mean_absolute_error: 3.1424 - val_loss: 9.7748 - val_mean_squared_error: 9.7748 - val_rms

10/10 [==============================] - 0s 23ms/step - loss: 15.2742 - mean_squared_error: 15.2742 - rmse: 3.9082 - mean_absolute_error: 3.0759 - val_loss: 10.0445 - val_mean_squared_error: 10.0445 - val_rmse: 3.1693 - val_mean_absolute_error: 2.3995 - lr: 2.5000e-04
Epoch 49/1000
10/10 [==============================] - 0s 25ms/step - loss: 15.8094 - mean_squared_error: 15.8094 - rmse: 3.9761 - mean_absolute_error: 3.1160 - val_loss: 9.3908 - val_mean_squared_error: 9.3908 - val_rmse: 3.0644 - val_mean_absolute_error: 2.3613 - lr: 2.5000e-04
Epoch 50/1000
10/10 [==============================] - 0s 28ms/step - loss: 15.0827 - mean_squared_error: 15.0827 - rmse: 3.8836 - mean_absolute_error: 3.0198 - val_loss: 9.7062 - val_mean_squared_error: 9.7062 - val_rmse: 3.1155 - val_mean_absolute_error: 2.3693 - lr: 2.5000e-04
Epoch 51/1000
10/10 [==============================] - 0s 30ms/step - loss: 15.5219 - mean_squared_error: 15.5219 - rmse: 3.9398 - mean_absolute_error: 3.0817 - val_loss

10/10 [==============================] - 0s 23ms/step - loss: 15.5047 - mean_squared_error: 15.5047 - rmse: 3.9376 - mean_absolute_error: 3.0780 - val_loss: 9.7786 - val_mean_squared_error: 9.7786 - val_rmse: 3.1271 - val_mean_absolute_error: 2.3578 - lr: 2.5000e-04
Epoch 78/1000
10/10 [==============================] - 0s 21ms/step - loss: 14.7735 - mean_squared_error: 14.7735 - rmse: 3.8436 - mean_absolute_error: 3.0205 - val_loss: 9.6378 - val_mean_squared_error: 9.6378 - val_rmse: 3.1045 - val_mean_absolute_error: 2.3523 - lr: 2.5000e-04
Epoch 79/1000
10/10 [==============================] - 0s 23ms/step - loss: 15.2600 - mean_squared_error: 15.2600 - rmse: 3.9064 - mean_absolute_error: 3.0366 - val_loss: 9.6209 - val_mean_squared_error: 9.6209 - val_rmse: 3.1018 - val_mean_absolute_error: 2.3443 - lr: 2.5000e-04
Epoch 80/1000
10/10 [==============================] - 0s 32ms/step - loss: 14.5421 - mean_squared_error: 14.5421 - rmse: 3.8134 - mean_absolute_error: 3.0128 - val_loss: 

10/10 [==============================] - 0s 32ms/step - loss: 14.0627 - mean_squared_error: 14.0627 - rmse: 3.7500 - mean_absolute_error: 2.9337 - val_loss: 9.1988 - val_mean_squared_error: 9.1988 - val_rmse: 3.0329 - val_mean_absolute_error: 2.3035 - lr: 1.2500e-04
Epoch 106/1000
10/10 [==============================] - 0s 28ms/step - loss: 14.2733 - mean_squared_error: 14.2733 - rmse: 3.7780 - mean_absolute_error: 2.9360 - val_loss: 9.8590 - val_mean_squared_error: 9.8590 - val_rmse: 3.1399 - val_mean_absolute_error: 2.3424 - lr: 1.2500e-04
Epoch 107/1000
10/10 [==============================] - 0s 34ms/step - loss: 14.3801 - mean_squared_error: 14.3801 - rmse: 3.7921 - mean_absolute_error: 2.9299 - val_loss: 9.2672 - val_mean_squared_error: 9.2672 - val_rmse: 3.0442 - val_mean_absolute_error: 2.3074 - lr: 1.2500e-04
Epoch 108/1000
10/10 [==============================] - 0s 23ms/step - loss: 13.8702 - mean_squared_error: 13.8702 - rmse: 3.7243 - mean_absolute_error: 2.9284 - val_los

10/10 [==============================] - 0s 22ms/step - loss: 13.2785 - mean_squared_error: 13.2785 - rmse: 3.6440 - mean_absolute_error: 2.8272 - val_loss: 9.2817 - val_mean_squared_error: 9.2817 - val_rmse: 3.0466 - val_mean_absolute_error: 2.2670 - lr: 1.2500e-04
Epoch 135/1000
10/10 [==============================] - 0s 22ms/step - loss: 13.7560 - mean_squared_error: 13.7560 - rmse: 3.7089 - mean_absolute_error: 2.9135 - val_loss: 9.1977 - val_mean_squared_error: 9.1977 - val_rmse: 3.0328 - val_mean_absolute_error: 2.2616 - lr: 1.2500e-04
Epoch 136/1000
10/10 [==============================] - 0s 21ms/step - loss: 14.4090 - mean_squared_error: 14.4090 - rmse: 3.7959 - mean_absolute_error: 3.0087 - val_loss: 9.3936 - val_mean_squared_error: 9.3936 - val_rmse: 3.0649 - val_mean_absolute_error: 2.2701 - lr: 1.2500e-04
Epoch 137/1000
10/10 [==============================] - 0s 22ms/step - loss: 13.4289 - mean_squared_error: 13.4289 - rmse: 3.6645 - mean_absolute_error: 2.8611 - val_los

Epoch 162/1000
10/10 [==============================] - 0s 19ms/step - loss: 13.5971 - mean_squared_error: 13.5971 - rmse: 3.6874 - mean_absolute_error: 2.8778 - val_loss: 9.3419 - val_mean_squared_error: 9.3419 - val_rmse: 3.0565 - val_mean_absolute_error: 2.2464 - lr: 3.1250e-05
Epoch 163/1000
10/10 [==============================] - 0s 19ms/step - loss: 13.4746 - mean_squared_error: 13.4746 - rmse: 3.6708 - mean_absolute_error: 2.8901 - val_loss: 9.3343 - val_mean_squared_error: 9.3343 - val_rmse: 3.0552 - val_mean_absolute_error: 2.2454 - lr: 3.1250e-05
Epoch 164/1000
10/10 [==============================] - 0s 22ms/step - loss: 13.0041 - mean_squared_error: 13.0041 - rmse: 3.6061 - mean_absolute_error: 2.8025 - val_loss: 9.3613 - val_mean_squared_error: 9.3613 - val_rmse: 3.0596 - val_mean_absolute_error: 2.2476 - lr: 3.1250e-05
Epoch 165/1000
10/10 [==============================] - 0s 21ms/step - loss: 13.3266 - mean_squared_error: 13.3266 - rmse: 3.6506 - mean_absolute_error: 2

10/10 [==============================] - 0s 42ms/step - loss: 13.1148 - mean_squared_error: 13.1148 - rmse: 3.6214 - mean_absolute_error: 2.8246 - val_loss: 9.2203 - val_mean_squared_error: 9.2203 - val_rmse: 3.0365 - val_mean_absolute_error: 2.2244 - lr: 1.5625e-05
Epoch 191/1000
10/10 [==============================] - 0s 27ms/step - loss: 13.1595 - mean_squared_error: 13.1595 - rmse: 3.6276 - mean_absolute_error: 2.8150 - val_loss: 9.3409 - val_mean_squared_error: 9.3409 - val_rmse: 3.0563 - val_mean_absolute_error: 2.2365 - lr: 1.5625e-05
Epoch 192/1000
10/10 [==============================] - 0s 31ms/step - loss: 12.9966 - mean_squared_error: 12.9966 - rmse: 3.6051 - mean_absolute_error: 2.8120 - val_loss: 9.2885 - val_mean_squared_error: 9.2885 - val_rmse: 3.0477 - val_mean_absolute_error: 2.2313 - lr: 1.5625e-05
Epoch 193/1000
10/10 [==============================] - 0s 32ms/step - loss: 13.1174 - mean_squared_error: 13.1174 - rmse: 3.6218 - mean_absolute_error: 2.8245 - val_los

10/10 [==============================] - ETA: 0s - loss: 12.8865 - mean_squared_error: 12.8865 - rmse: 3.5898 - mean_absolute_error: 2.8137
Epoch 00218: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
10/10 [==============================] - 0s 24ms/step - loss: 12.8865 - mean_squared_error: 12.8865 - rmse: 3.5898 - mean_absolute_error: 2.8137 - val_loss: 9.2408 - val_mean_squared_error: 9.2408 - val_rmse: 3.0399 - val_mean_absolute_error: 2.2239 - lr: 7.8125e-06


NameError: name 'boom' is not defined